In [ ]:
# ✅ Install geopy if not yet installed
!pip install geopy

import pandas as pd
from geopy.geocoders import Nominatim
import time
from functools import lru_cache


# STEP 1: LOAD RAW DATA
file_path = "job_ads.csv"  # Make sure your CSV is tab-separated (\t)
df = pd.read_csv(file_path, sep="\t", encoding='utf-8', on_bad_lines='skip')

# STEP 2: CLEAN ENCODING ISSUES
replacements = {
    "‚Äì": "-", "√©": "é", "√†": "à", "√º": "ü", "√¢": "â",
    "√§": "ä", "√¶": "ö", "√ä": "ä", "√": ""
}
df.replace(replacements, regex=True, inplace=True)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# STEP 3: GEOCODE LOCATIONS
geolocator = Nominatim(user_agent="job_mapper")

@lru_cache(maxsize=500)
def get_coordinates_cached(location):
    try:
        loc = geolocator.geocode(location + ", Switzerland")
        time.sleep(1)  # pause BETWEEN requests to avoid blocking
        if loc:
            return pd.Series([loc.latitude, loc.longitude])
    except Exception as e:
        print(f"Error geocoding {location}: {e}")
    return pd.Series([None, None])

df[['latitude', 'longitude']] = df['location'].apply(get_coordinates_cached)

# STEP 4: SAVE OUTPUT
df.to_csv("cleaned_jobs_with_geo.csv", index=False)
print("✅ Saved as 'cleaned_jobs_with_geo.csv'")

/var/folders/sf/3_6zfybs6_xcm9ncfcfxk_zr0000gn/T/ipykernel_18983/2038008961.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


KeyError: 'location'